## Importing Libraries 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from DataSet import DataSet as DS
%matplotlib qt 

ModuleNotFoundError: No module named 'pandas'

## Analytical Functions for $\vec{A}$ and $\vec{B}$ components
$\vec{A}(x,y,z) = \{x y z, x y^{2} z, x y^{3} z^{2}\}$

$\vec{B}(x,y,z) = \nabla \times \vec{A}(x,y,z) = \{3 x y^{2} z^{2}-x y^{2},x y-y^{3} z^{2},y^{2} z-x z\}$

These function will be used to estimate $\vec{A}$ and $\vec{B}$ over __Edges__ and __Cell-Faces__, respectively

In [ ]:
def Ax(Coords: dict) -> np.ndarray:
    return Coords["X"]*Coords["Y"]*Coords["Z"]


def Ay(Coords: dict) -> np.ndarray:
    return Coords["X"]*Coords["Y"]*Coords["Y"]*Coords["Z"]


def Az(Coords: dict) -> np.ndarray:
    return Coords["X"] * Coords["Y"] * Coords["Y"] * Coords["Y"] * \
            Coords["Z"] * Coords["Z"]


def Bx(Coords: dict) -> np.ndarray:
    return -Coords["X"] * Coords["Y"] * Coords["Y"] + \
        3 * Coords["X"] * Coords["Y"] * Coords["Y"] * \
             Coords["Z"] * Coords["Z"]


def By(Coords: dict) -> np.ndarray:
    return Coords["X"] * Coords["Y"] - \
        Coords["Y"]**3 * Coords["Z"]**2


def Bz(Coords: dict) -> np.ndarray:
    return -Coords["X"] * Coords["Z"] + \
        Coords["Y"]**2 * Coords["Z"]


## Creating a Mesh Structure

In [ ]:
nx, ny, nz = 40, 40, 40  # Number of Cells in x-, y-, and z-direction
x0, y0, z0 = -1.0, -1.0, -1.0  # starting values of mesh in x-, y-, and z-direction
x1, y1, z1 = 1.0, 1.0, 1.0  # ending values of mesh in x-, y-, and z-direction

ds1 = DS(SystemOfCoords="CAR", NCell=(nx, ny, nz), startval=(x0, y0, z0), endval=(x1, y1, z1))

## Evaluating $\vec{A}$ and $\vec{B}$ using predefined functions (on Edges and Cell-Faces)

In [ ]:
ds1.Scalar("Ax", "EdgeX", Ax)
ds1.Scalar("Ay", "EdgeY", Ay)
ds1.Scalar("Az", "EdgeZ", Az)

ds1.Scalar("BEx", "FaceX", Bx)
ds1.Scalar("BEy", "FaceY", By)
ds1.Scalar("BEz", "FaceZ", Bz)

## Building numerical results for $\vec{B}$ to compare with exact vaulues to check if "CurlEdgeToFace" method works as expected

In [ ]:
ds1.CurlEdgeToFace("Ax", "Ay", "Az", "Bx", "By", "Bz")
plt.ion()
ErrorZ = ds1.vars["Bz"]['val'] - ds1.vars["BEz"]['val']

# Loop to check the entire computational box
# ---<( Smaller values are better )>---
plt.ion()
for k in range(nz):
    plt.figure()
    plt.imshow((ErrorZ[:,:,k] / ds1.vars["BEz"]['val'][:,:,k]).T, origin='lower left')
    plt.colorbar()
    plt.title("k={0:02}".format(k))
    plt.pause(2)
    #input("Press Enter...")
    plt.close()

## Now extract Vector potential and reproduce $\vec{B}$ to check if extraction method works as expected by compari with exact vaulues

In [ ]:
ds1.ExtractAFromBFace("BEx", "BEy", "BEz", "AEx", "AEy", "AEz")
ds1.CurlEdgeToFace("AEx", "AEy", "AEz", "BBBx", "BBBy", "BBBz")

In [ ]:
ErrorX = ds1.vars["BBBx"]['val'] - ds1.vars["BEx"]['val']
for k in range(nx):
    plt.figure()
    plt.imshow((ErrorX[:,:,k] / ds1.vars["BEx"]['val'][:,:,k]).T, origin='lower left')
    plt.colorbar()
    plt.title(r"$B_{x}$ "+" k={0:02}".format(k))
    plt.pause(1)
    plt.close()

In [ ]:
ErrorY = ds1.vars["BBBy"]['val'] - ds1.vars["BEy"]['val']
for k in range(ny):
    plt.figure()
    plt.imshow((ErrorY[:,:,k] / ds1.vars["BEy"]['val'][:,:,k]).T, origin='lower left')
    plt.colorbar()
    plt.title(r"$B_{y}$ "+" k={0:02}".format(k))
    plt.pause(1)
    plt.close()

In [ ]:
ErrorZ = ds1.vars["BBBz"]['val'] - ds1.vars["BEz"]['val']
for k in range(nz):
    plt.figure()
    plt.imshow((ErrorZ[:,:,k] / ds1.vars["BEz"]['val'][:,:,k]).T, origin='lower left')
    plt.colorbar()
    plt.title(r"$B_{z}$ "+" k={0:02}".format(k))
    plt.pause(1)
    plt.close()